In [8]:
import numpy as np
import pandas as pd
import cv2 as cv
import sys
import string
import os
import pickle
import sys

sys.path.append('..')

from kp_package.annotation_tool import *
from kp_package.annotation_display import see_annot
from kp_package.account_management import start, stop
from kp_package.io import write




cont = True
lock = False
dirty = False


while(cont):
    
    # 1.0 Checking Account Configuration and Data Lock
    
    print("Checking Account Configuration...")
    io_ok, lock, config_ok, annotator_id, annotator, config = start()
    image_folder = config.get('im_pth', '../../../Images')
    cid = config.get('cid',0)
    
    if(not(lock)):
        print("Could not Obtain Lock. Close Other operations First. Terminating...")
        break

    if(not(io_ok)):
        print("Account Not configured. Please Obtain Configuration File from Moderator")
        break

    if(not(config_ok)):
        print("Image path not set. Run set_image_path first")
        break

    print("Welcome " + annotator)


    # 2.0 Setting path for data files
    
    data_file_folder = '../../annotator_data'
    to_annotate_file = 'to_annotate_{}'.format(annotator_id) + '.csv'
    keypoints_file = 'KEYPOINTS_DATASET_{}'.format(annotator_id) + '.csv'
    status_file = 'STATUS_{}'.format(annotator_id) + '.csv'

    data_file_path = os.path.join(data_file_folder, keypoints_file)
    status_file_path = os.path.join(data_file_folder, status_file)
    to_annotate_path = os.path.join(data_file_folder, to_annotate_file)

    # 3.0 Setting GUI Window Size
    
    window_size = (1028,668)


    ########


    # 4.0 Reading List of allocated images
    
    if(os.path.exists(to_annotate_path)):
      #If list of images to be annotated has been provided
      tmp = pd.read_csv(to_annotate_path)
      tmp = tmp[tmp['annotator_id'] == annotator_id]
      file_names = tmp['img_list'].values
    else:
      print("No to_annotate list available ")
      break


    # 5.0 Loading Keypoints dataset and status files
    
    if(os.path.exists(data_file_path)):
      #If some annotations are already done, load them
      data = pd.read_csv(data_file_path, index_col = False)
    else:
      #Create a new empty dataframe to store annotations and save it to a file
      data = pd.DataFrame(columns = ['id', 'pid', 'type', 'x', 'y', 'attr', 'person', 'img_id', 'hidden', 'x1', 'y1'])

      stat = write('pandas', data, data_file_path, create = True)
    
      if(not stat):
        break
      #Delete after check # data.to_csv(data_file_path, index = False)

    data_backup = data.copy()

    if(os.path.exists(status_file_path)):
      #If status file exists, load it
      completed = pd.read_csv(status_file_path, index_col = False)
    else:
      #Else create a new empty datasframe and save it to a file. In this code Dataframe completed corresponds to status file
      completed = pd.DataFrame(columns = ['file_name','annotator','success', 'sent_for_review', 'accepted', 'reviewer_remarks', 'reviewer', 'expunge'])

      stat = write('pandas', completed, status_file_path, create = True)
      if(not stat):
        break

      # Delete after check # completed.to_csv(status_file_path, index = False)

    completed_backup = completed.copy()


   
    if (len(file_names) < 1):
        print("No more images to annotate")
        break

    # 6.0 Main loop over allocated images for annotation
    
    for file_name in file_names:
        
      # 6.1 To skip already annotated images. Only considers those entries which were completed successfully 
      completed_files = completed[completed['success'] == True]
      completed_files = completed_files['file_name'].values
      all_files = completed['file_name'].values
      expunged_files = completed[completed['expunge'] == True]
      expunged_files = expunged_files['file_name'].values

      # 6.2 To skip annotated images and non image files
      if(file_name  in completed_files) or (file_name in expunged_files) or (not (file_name.endswith('.png'))):
        continue

      file_path = os.path.join(image_folder, file_name)

      if(not (os.path.exists(file_path))):
            print("Image for " + file_name + " not found. Ignoring")
            continue

      All_annotations = Final_Annotation()

      img = cv.imread(file_path)

      # 6.3 If annotations exist for this image, display them first
      if(file_name in all_files):
          see_annot(data_file_path, status_file_path, window_size, image_folder, data_file_folder, file_name)

      # 6.4 Reload keypoints dataset and status dataset as they might have been changed by see_annot
      data = pd.read_csv(data_file_path, index_col = False)
      completed = pd.read_csv(status_file_path, index_col = False)

      this_rec = data[data['img_id'] == file_name]

      #6.5 Setting unique id for next person to be annotated
      if(len(this_rec) == 0):
            next_id = 0
      else:
          num = this_rec['person'].values
          if(len(num) == 0):
            next_id = 0
          else:
            next_id = max(num) + 1

      # 6.6 Annotation code proper
      added = tool_GUI(file_name, All_annotations, name = file_name, img = img, window_size = window_size, next_id = next_id)

      new_file = pd.DataFrame({'file_name':[file_name], 'annotator':[annotator], 'success':[False], 'sent_for_review':[False], 'accepted':[False], 'reviewer_remarks':["Nil"], 'reviewer':["Nil"], 'expunge':[False]})
      if(file_name not in all_files):
        completed = completed.append(new_file, ignore_index = True)
        dirty = True

      stat = write('pandas', completed, status_file_path)
      if(not stat):
        break

      #Delete after Check # completed.to_csv(status_file_path, index = False) 
      dirty = False




      # 6.7 Adding annotations made by tool_GUI to "data"
      if(added):
          a = All_annotations.annotations
          add = pd.DataFrame(a)

          data = data.append(add, ignore_index = True)
          dirty = True

          stat = write('pandas', data, data_file_path)
          if(not stat):
            break

          '''tmp_path = os.path.join(data_file_folder, 'tmp_keypoints_dataset.csv')
          data.to_csv(tmp_path, index = False)

          os.remove(data_file_path)
          os.rename(tmp_path,data_file_path)'''
          dirty = False

          inp = input("Want to review annotations for this image? (y/n) ")
          if(inp == 'y') or (inp == 'Y'):
                see_annot(data_file_path, status_file_path, window_size, image_folder, data_file_folder, file_name)
                #Reload keypoints dataset and status dataset as they might have been changed by see_annot
                data = pd.read_csv(data_file_path, index_col = False)
                completed = pd.read_csv(status_file_path, index_col = False)

          q = input('Mark this image as completed? You will not be able to annotate this image further if marked completed (y/n) ')
          if(q == 'y') or (q == 'Y'):
                    this_record = completed['file_name'] == file_name
                    completed.loc[this_record,'success'] = True
                    dirty = True
                    print("Image " + file_name + " completed")
          else:
                    print("Image " + file_name + " kept as Incomplete. You can add further annotations when your run the code next time")



      # 6.8 Case when nothing is added by tool_GUI
      else:
        if(len(this_rec) == 0):
                        completed = completed[completed['file_name'] != file_name]
                        dirty = True
                        print("No remaining Annotations for {}. Status Record Expunged".format(file_name))
        else:            
            q = input('No annotation added. Still mark this image as completed? You will not be able to annotate this image further if marked completed (y/n) ')
            if(q == 'y') or (q == 'Y'):
                    this_record = completed['file_name'] == file_name
                    completed.loc[this_record,'success'] = True
                    dirty = True
                    print("Image " + file_name + " completed")
                    this_rec = data[data['img_id'] == file_name]


      stat = write('pandas', completed, status_file_path)
      if(not stat):
        break




      #Delete after check # completed.to_csv(status_file_path, index = False)
      dirty = False
      
      # 6.9 user input for going to next image in list
      inp = input("Enter 'y' to go to next image, 'n' to quit ")
      if(inp == 'y') or (inp == 'Y'):
        cont = True
        continue
      else:
        cont = False
        break

      print(completed['file_name'].values)
    
    #Exit outer loop. It is there only to avoid using sys.exit()
    cont = False




# 7.0 If any write operation failed due to csv file open, this will give one more chance to save the data
if(dirty):
    a = input("Some of the Write operations have failed. Pl close any open csv files and press enter to try again")
    stat = write('pandas', completed, status_file_path)
    if(not stat):
        print("Failed Again. Please check data integrity. You may try to save backup dataframes to avoid losing data")   

            
        
    stat = write('pandas', data, data_file_path)
    if(not stat):
        print("Failed Again. Please check data integrity. You may try to save backup dataframes to avoid losing data")   

    

# 8.0 Release Lock


if(lock):
        stop()
        print("Lock Released")
 

print("Session Complete")




Checking Account Configuration...
Welcome Dummy1
['Root', 'Forehead', 'Left Eye', 'Left Ear', 'Left Shoulder', 'Left Hip', 'Right Eye', 'Right Ear', 'Right Shoulder', 'Right Hip', 'Nose', 'Left Elbow', 'Left Knee', 'Right Elbow', 'Right Knee', 'Left Wrist', 'Left Ankle', 'Right Wrist', 'Right Ankle', 'Mouth', 'Crown', 'Vaahan', 'Article_Left', 'Article_Right']
Forehead Added with id 1
Nose Added with id 2


C:\Users\hp\AppData\Local\Temp\ipykernel_32632\2489859110.py:168: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  completed = completed.append(new_file, ignore_index = True)
C:\Users\hp\AppData\Local\Temp\ipykernel_32632\2489859110.py:186: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(add, ignore_index = True)


Want to review annotations for this image? (y/n) y
Mark this image as completed? You will not be able to annotate this image further if marked completed (y/n) y
Image Bhutan_I101.png completed
Enter 'y' to go to next image, 'n' to quit y


C:\Users\hp\AppData\Local\Temp\ipykernel_32632\2489859110.py:168: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  completed = completed.append(new_file, ignore_index = True)


No remaining Annotations for Bhutan_I102.png. Status Record Expunged
Enter 'y' to go to next image, 'n' to quit n
Lock Released
Session Complete


In [ ]:
#To be used only in if image window hangs
cv.destroyAllWindows()